# Assignment 1: Part of Speech Tagging with Bigram Hidden Markov Model

All the necessary libraries for opening and processing conllu files are imported

In [11]:
import numpy as np
import math
import pickle as pickle
from conllu import parse
from conll_df import conll_df
import nltk
from io import open
from collections import defaultdict
import pyconll as pcon
import itertools
import sys
import time
from nltk.stem import WordNetLemmatizer

## Part 1: Defining global variables

The global variables such as the tag sets and the name of the training corpuses are defined here

In [12]:
#Global variables for storing part of speech tags and maximum value of log probabilities
max_val = - math.log(1.0 / 80000)

tag_set = {'ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 
'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X'}

refined_tag_set = {',', 'POS', 'WP$', 'MD', 'GW', 'EX', 'NN', 
                   '-LRB-', 'RBR', 'WRB', 'JJR', 'XX', 'RB', 'NNP', 
                   'NNPS', 'RP', ':', 'JJ', 'CC', 'PRP', 'AFX', 'VB', 'WDT', 
                   '``', 'NNS', 'WP', 'IN', 'VBP', 'NFP', 'PRP$', '.', 'SYM', 
                   'DT', 'TO', 'VBZ', 'PDT', 'VBG', 'FW', 'CD', 'HYPH', 'VBD', 'VBN', 
                   '-RRB-', 'UH', 'LS', 'JJS', 'RBS', 'ADD', "''", '$'}

eng_train = 'en-ud-train.conllu' 
eng_dev = 'en-ud-dev.conllu'
eng_test = 'en-ud-test.conllu'

es_train = 'es-ud-train.conllu'
es_dev = 'es-ud-dev.conllu'
es_test = 'es-ud-test.conllu'

## Part 2: A processor to process sentences and tag lists

First a file processor class is defined to get all the sentences and their respective tags into a list

In [13]:
class FilePreprecessor:
	def __init__(self, train_file_name, dev_file_name, test_file_name, refined):
		self.train_file_name = train_file_name
		self.dev_file_name = dev_file_name
		self.test_file_name = test_file_name
		self.refined = refined
    
    #Obtain all sentences from the conllu file
	def obtain_sentences(self, file_name):
		sentences = pcon.load_from_file(file_name)
		return sentences

    #Process all the sentences and store them into a list with their respective tags
	def preprocess_sentences(self, sentences):
		data_list = []
		sentence_list = []
		full_tag_list = []
		for sentence in sentences:
			word_list = []
			tag_list = []
			for token in sentence:
				word_list.append(token.form)
				if self.refined:
					tag_list.append(token.xpos)
				else:
					tag_list.append(token.upos)
            #Convert all words in the list into lower case
			word_list = [str(x).lower() for x in word_list]
			p = nltk.PorterStemmer()
			wnl = WordNetLemmatizer()
			word_list = [wnl.lemmatize(x) for x in word_list]
			sentence_with_tags = [tuple(word_list), tuple(tag_list)]
			
			#Get the list of sentences, list of tags and the combined list
			sentence_list.append(word_list)
			full_tag_list.append(tag_list)
			data_list.append(tuple(sentence_with_tags))
		return sentence_list, full_tag_list, data_list


## Part 3: Implementation of the Viterbi Algorithm

The Viterbi algorithm is then implemented with the calculation of the probabilities and the final evaluation of the tags.

In [14]:
class ViterbiSolver:
	def __init__(self, refined=False):
    #Store different types of probabilities in dictionaries
		self.refined = refined
		self.start_prob = defaultdict()
		self.tag_set = refined_tag_set if self.refined==True else tag_set

		self.tag_prob = dict()
		for tag in self.tag_set:
			self.tag_prob[tag] = 0

		self.transition_prob = defaultdict(dict)
		for row_tag in self.tag_set:
			for column_tag in self.tag_set:
				self.transition_prob[row_tag][column_tag] = 0

		self.emission_prob = defaultdict(dict)
    
    #Static method to normalize a dictionary based on the log of all probabilities
	@staticmethod
	def normalize_dict(numeric_dict):
		dict_length = len(set(numeric_dict))
		total_log = math.log(sum(numeric_dict.values()) + 0.1*dict_length)
		for key, value in numeric_dict.items():
			if value == 0:
				numeric_dict[key] = sys.maxsize 
			else:
				numeric_dict[key] = total_log/math.log(value + 0.1)
    
    #Calculate the different types of probabilities based on 
    #the list of sentences and tags
	def evaluate_probabilities(self, data_list):
		for sentence, tags in data_list:
			if tags[0] not in self.start_prob and tags[0] != None:
				self.start_prob[tags[0]] = 0.1
			if tags[0] != None:
				self.start_prob[tags[0]] += 1.1

			for index in range(0, len(tags) - 1):
				current_tag = tags[index]
				next_tag = tags[index + 1]
				if current_tag != None and next_tag != None: 
					self.transition_prob[current_tag][next_tag] += 1.1

			for word, tag in zip(sentence, tags):
				if tag != None:
					self.tag_prob[tag] += 1.1
				if tag not in self.emission_prob[word] and tag != None:
					self.emission_prob[word][tag] = 0.1
				if tag != None:
					self.emission_prob[word][tag] += 1.1
        
        #Normalize the dictionaries
		self.normalize_dict(self.start_prob)
		
		self.normalize_dict(self.tag_prob)

		for dict_of_tag in self.transition_prob.values():
			self.normalize_dict(dict_of_tag)

		for dict_of_tag in self.emission_prob.values():
			for tag, count in dict_of_tag.items():
				if tag != None:
					if count >= 1:
						dict_of_tag[tag] = (math.log(self.tag_prob[tag] + 0.1)
                                         /math.log(count + 0.1))  
					else:
						dict_of_tag[tag] = sys.maxsize
    
    #Obtain the emission probabilities for each word and store them in a dictionary
	def obtain_emission_probabilities(self, word):
		emission_dict_of_probs = defaultdict() 
		for tag in self.tag_set:
			if word in self.emission_prob and tag in self.emission_prob[word] and tag != None:
				emission_dict_of_probs[tag] = self.emission_prob[word][tag]
			else:
				emission_dict_of_probs[tag] = max_val
		return emission_dict_of_probs

    #Implementation of the Viterbi algorithm using a 2-D numpy array as the Viterbi matrix
	def viterbi(self, sentence):
		'''params sentence: a list of different words in the sentence
		return output_tags: a list of the tags'''
		list_of_tags = list(self.tag_set)
		num_rows = len(list_of_tags)
		num_cols = len(sentence)
		viterbi_matrix = np.empty((num_rows, num_cols), dtype=tuple)
        
        #Go through each column to update the probability
		for column_index, current_word in enumerate(sentence):
			current_emission_probs = self.obtain_emission_probabilities(current_word)
			for row_index, current_tag in enumerate(list_of_tags):
				if column_index == 0:
					if current_word in self.start_prob:
						start_prob = self.start_prob[current_tag]
					else:
						start_prob = max_val
					viterbi_matrix[row_index][column_index] = (-1, current_emission_probs[current_tag]*start_prob)

				else:
					best_tuple = (-1, max_val)
					for prev_row_index, prev_tag in enumerate(list_of_tags):
						prev_prob = viterbi_matrix[prev_row_index][column_index - 1][1]
						current_prob = prev_prob*current_emission_probs[current_tag]*self.transition_prob[prev_tag][current_tag]
						if current_prob < best_tuple[1]:
							best_tuple = (prev_row_index, current_prob)

					viterbi_matrix[row_index][column_index] = best_tuple
        
        #Go through the last column to find the index of the maximum probability
		(max_index, max_prob) = (-1, max_val)
		for row in range(num_rows):
			current_prob = viterbi_matrix[row][num_cols - 1][1] 
			if current_prob < max_prob:
				(max_index, max_prob) = (row, current_prob)
        
        #Backtrack to obtain the best sequence of tags in the columns
		output_tags = list()
		for col in range(num_cols - 1, 0, -1):
			output_tags.insert(0, list_of_tags[max_index])
			max_index = viterbi_matrix[max_index][col][0]
		output_tags.insert(0, list_of_tags[max_index])
		return output_tags
    
    #Method to score the performance of the algorithm on the test corpus
	def score_test_corpus(self, raw_sentences, raw_tags):
		all_viterbi_tags = []
		for sentence in raw_sentences:
			sentence_tag = self.viterbi(sentence)
			all_viterbi_tags.append(sentence_tag)

		final_viterbi_tags = list(itertools.chain.from_iterable(all_viterbi_tags))
		final_test_tags = list(itertools.chain.from_iterable(raw_tags))
		total_matches = sum([1 for i, j in zip(final_viterbi_tags, final_test_tags) if i == j])
		accuracy = total_matches/len(final_test_tags)
		return total_matches, accuracy

## Part 4: English Part of Speech Tagging

Here is the English portion of the assignment. A sample sentence is obtained and then decoded using the Viterbi algorithm. The accuracy on the development and the test sets are then reported. The corresponding dictionary files are also saved into pickle files. The accuracy of the algorithm on the English test set is 84.4%.

In [15]:
#English portion: Load the preprocessor and obtain the sentence with tags
eng_preprocessor = FilePreprecessor(eng_train, eng_dev, eng_test, refined=False)

eng_train_data = eng_preprocessor.obtain_sentences(eng_train)

_, _, train_data_list = eng_preprocessor.preprocess_sentences(eng_train_data)

#Calculate the probabilities using the viterbi solver
eng_solver = ViterbiSolver(refined=False)

eng_solver.evaluate_probabilities(train_data_list)

#Get a sample sentence and decode it with Viterbi
eng_sample_sentence  = ['I', 'like', 'NLP', 'very', 'much']

eng_sample_tags = eng_solver.viterbi(eng_sample_sentence)

print('Tags for the sample English sentence: {}'.format(eng_sample_tags))


#Obtain the dictionaries of transition and emission probabilities and 
#save them to pickle files
eng_transition = eng_solver.transition_prob 

eng_emission = eng_solver.emission_prob

eng_dicts = [eng_transition, eng_emission]

with open('eng_probabilities.pkl', 'wb') as eng_prob:
	pickle.dump(eng_dicts, eng_prob)


#Evaluate the performance of the algorithm on the development set
dev_data = eng_preprocessor.obtain_sentences(eng_dev)

dev_sentences_list, dev_tags_list, _ = eng_preprocessor.preprocess_sentences(dev_data)

total_dev_matches, dev_accuracy = eng_solver.score_test_corpus(dev_sentences_list, 
                                                               dev_tags_list)
print('The accuracy on the English development set is : {:.3f} %'
      .format(dev_accuracy*100.0))


#Test the performance of the algorithm on the test set
test_data = eng_preprocessor.obtain_sentences(eng_test)

test_sentences_list, test_tags_list, _ = eng_preprocessor.preprocess_sentences(test_data)

total_matches, accuracy = eng_solver.score_test_corpus(test_sentences_list, 
                                                       test_tags_list)
print('The accuracy on the English test set is : {:.3f} %'
      .format(accuracy*100.0))

Tags for the sample English sentence: ['ADJ', 'ADJ', 'AUX', 'ADV', 'ADJ']
The accuracy on the English development set is : 84.553 %
The accuracy on the English test set is : 84.365 %


## Part 5: Spanish Part of Speech Tagging 

Here is the Spanish portion of the assignment. A sample Spanish sentence is obtained and then decoded using the Viterbi algorithm. The accuracy on the development and the test sets are then also reported. The Spanish probability dictionaries are also saved into pickle files. The accuracy of the algorithm on the Spanish test set is 88.2%.

In [17]:
#Spanish portion: Load the processor and obtain the sentence with tags
es_preprocessor = FilePreprecessor(es_train, es_dev, es_test, refined=False)

es_train_data = es_preprocessor.obtain_sentences(es_train)

_, _, es_train_data_list = es_preprocessor.preprocess_sentences(es_train_data)


#Calculate the probabilities using the viterbi solver
es_solver = ViterbiSolver(refined=False)

es_solver.evaluate_probabilities(es_train_data_list)


#Get a sample Spanish sentence and decode it with Viterbi
es_sample_sentence  = ['Yo', 'te', 'quiero', 'demasiado', 'mucho']

es_sample_tags = es_solver.viterbi(es_sample_sentence)

print('Tags for the sample Spanish sentence: {}'.format(es_sample_tags))


#Obtain the dictionaries of transition and emission probabilities 
#and save them to pickle files
es_transition = es_solver.transition_prob 

es_emission = es_solver.emission_prob

es_dicts = [es_transition, es_emission]

with open('es_probabilities.pkl', 'wb') as es_prob:
	pickle.dump(es_dicts, es_prob)

    
#Evaluate the performance of the algorithm on the development set
es_dev_data = es_preprocessor.obtain_sentences(es_dev)

es_dev_sentences_list, es_dev_tags_list, _ = es_preprocessor.preprocess_sentences(es_dev_data)

es_total_dev_matches, es_dev_accuracy = es_solver.score_test_corpus(es_dev_sentences_list, 
                                              es_dev_tags_list)
print('The accuracy on the Spanish development set is : {:.3f} %'
      .format(es_dev_accuracy*100.0))


#Test the performance of the algorithm on the test set
es_test_data = es_preprocessor.obtain_sentences(es_test)

es_test_sentences_list, es_test_tags_list, _ = es_preprocessor.preprocess_sentences(es_test_data)

es_total_matches, es_accuracy = es_solver.score_test_corpus(es_test_sentences_list, 
                                          es_test_tags_list)
print('The accuracy on the Spanish test set is : {:.3f} %'.format(es_accuracy*100.0))

Tags for the sample Spanish sentence: ['ADJ', 'AUX', 'VERB', 'PRON', 'PRON']
The accuracy on the Spanish development set is : 88.315 %
The accuracy on the Spanish test set is : 88.335 %


## Part 6: English Part of Speech Tagging with Refined Tag Sets

The probabilities are calculated for the part of speech tagging using refined tag sets. An English sentence is used to test the tagging using the refined tag set. 

In [ ]:
#Refined tag set portion
ref_preprocessor = FilePreprecessor(eng_train, eng_dev, eng_test, refined=True)

ref_train_data = ref_preprocessor.obtain_sentences(eng_train)

_, _, ref_train_data_list = ref_preprocessor.preprocess_sentences(ref_train_data)


#Calculate the probabilities using the viterbi solver
ref_solver = ViterbiSolver(refined=True)

ref_solver.evaluate_probabilities(ref_train_data_list)

#Get a sample sentence and decode it with Viterbi
ref_sample_sentence  = ['I', 'like', 'NLP', 'very', 'much']

ref_sample_tags = ref_solver.viterbi(ref_sample_sentence)

print('Tags for the English sentence using the refined tag set: {}'
      .format(ref_sample_tags))

#Obtain the dictionaries of transition and emission probabilities 
#and save them to pickle files
ref_transition = ref_solver.transition_prob 

ref_emission = ref_solver.emission_prob

ref_dicts = [ref_transition, ref_emission]

with open('ref_probabilities.pkl', 'wb') as ref_prob:
	pickle.dump(ref_dicts, ref_prob)

## Part 7: Error Analysis and Suggested Solutions to Challenges

The accuracy of the algorithm ranges from 84-90% for both the English and the Spanish corpus. This is a bit low in comparison with the standard baseline of 90-95% accuracy. To improve on this performance various smoothing techniques can be used in the calculation of the emission and transition probabilities for each of the corpus, such as the Laplace smoothing (adding one to the word count and adding the total number of distinct words to the total number of words in the corpus) in the final probability calculation. Assigning a small probability to an unseen word can also improve the accuracy of the probability calculation process.